# Amazon SageMaker Batch Transform: Associate prediction results with their corresponding input records


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

---

_**Use SageMaker's XGBoost to train a binary classification model and for a list of tumors in batch file, predict if each is malignant**_

_**It also shows how to use the input output joining / filter feature in Batch transform in details**_

---



## Background
This purpose of this notebook is to train a model using SageMaker's XGBoost and UCI's breast cancer diagnostic data set to illustrate at how to run batch inferences and how to use the Batch Transform I/O join feature. UCI's breast cancer diagnostic data set is available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give training and batch transform access to your data. The snippet below will use the same role used by your SageMaker notebook instance. Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.

In [32]:
import os
import boto3
import sagemaker

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
role = 'arn:aws:iam::898487103080:role/ml-experiments'

bucket = 'turin-complete-experiments'
prefix = "breast-cancer"

---
## Data sources

> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

## Data preparation


Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [6]:
import pandas as pd
import numpy as np

data = pd.read_csv('../datasets/wdbc.csv')

# specify columns extracted from wbdc.names
selected_columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

data = data[selected_columns]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
405,904971,B,10.940,18.59,70.39,370.0,0.10040,0.07460,0.04944,0.029320,...,12.400,25.58,82.76,472.4,0.1363,0.16440,0.14120,0.07887,0.2251,0.07732
316,894090,B,12.180,14.08,77.25,461.4,0.07734,0.03212,0.01123,0.005051,...,12.850,16.47,81.60,513.1,0.1001,0.05332,0.04116,0.01852,0.2293,0.06037
75,8610404,M,16.070,19.65,104.10,817.7,0.09168,0.08424,0.09769,0.066380,...,19.770,24.56,128.80,1223.0,0.1500,0.20450,0.28290,0.15200,0.2650,0.06387
47,85715,M,13.170,18.66,85.98,534.6,0.11580,0.12310,0.12260,0.073400,...,15.670,27.95,102.80,759.4,0.1786,0.41660,0.50060,0.20880,0.3900,0.11790
553,924342,B,9.333,21.94,59.01,264.0,0.09240,0.05605,0.03996,0.012820,...,9.845,25.05,62.86,295.8,0.1103,0.08298,0.07993,0.02564,0.2435,0.07393
241,883539,B,12.420,15.04,78.61,476.5,0.07926,0.03393,0.01053,0.011080,...,13.200,20.37,83.85,543.4,0.1037,0.07776,0.06243,0.04052,0.2901,0.06783
492,914062,M,18.010,20.56,118.40,1007.0,0.10010,0.12890,0.11700,0.077620,...,21.530,26.06,143.40,1426.0,0.1309,0.23270,0.25440,0.14890,0.3251,0.07625
166,87127,B,10.800,9.71,68.77,357.6,0.09594,0.05736,0.02531,0.016980,...,11.600,12.02,73.66,414.0,0.1436,0.12570,0.10470,0.04603,0.2090,0.07699


#### Key observations:
* The data has 569 observations and 32 columns.
* The first field is the 'id' attribute that we will want to drop before batch inference and add to the final inference output next to the probability of malignancy.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features that we will use for training and inferencing.

Let's replace the M/B diagnosis with a 1/0 boolean value. 

In [7]:
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
458,9112594,0,13.00,25.13,82.61,520.2,0.08369,0.05073,0.012060,0.017620,...,14.34,31.88,91.06,628.5,0.1218,0.10930,0.04462,0.05921,0.2306,0.06291
447,9110944,0,14.80,17.66,95.88,674.8,0.09179,0.08890,0.040690,0.022600,...,16.43,22.74,105.90,829.5,0.1226,0.18810,0.20600,0.08308,0.3600,0.07285
454,911202,0,12.62,17.15,80.62,492.9,0.08583,0.05430,0.029660,0.022720,...,14.34,22.15,91.62,633.5,0.1225,0.15170,0.18870,0.09851,0.3270,0.07330
241,883539,0,12.42,15.04,78.61,476.5,0.07926,0.03393,0.010530,0.011080,...,13.20,20.37,83.85,543.4,0.1037,0.07776,0.06243,0.04052,0.2901,0.06783
334,897374,0,12.30,19.02,77.88,464.4,0.08313,0.04202,0.007756,0.008535,...,13.35,28.46,84.53,544.3,0.1222,0.09052,0.03619,0.03983,0.2554,0.07207
490,91376701,0,12.25,22.44,78.18,466.5,0.08192,0.05200,0.017140,0.012610,...,14.17,31.99,92.74,622.9,0.1256,0.18040,0.12300,0.06335,0.3100,0.08203
50,857343,0,11.76,21.60,74.72,427.9,0.08637,0.04966,0.016570,0.011150,...,12.98,25.72,82.98,516.5,0.1085,0.08615,0.05523,0.03715,0.2433,0.06563
235,88249602,0,14.03,21.25,89.79,603.4,0.09070,0.06945,0.014620,0.018960,...,15.33,30.28,98.27,715.5,0.1287,0.15130,0.06231,0.07963,0.2226,0.07617


Let's split the data as follows: 80% for training, 10% for validation and let's set 10% aside for our batch inference job. In addition, let's drop the 'id' field on the training set and validation set as 'id' is not a training feature. For our batch set however, we keep the 'id' feature. We'll want to filter it out prior to running our inferences so that the input data features match the ones of training set and then ultimately, we'll want to join it with inference result. We are however dropping the diagnosis attribute for the batch set since this is what we'll try to predict.

In [8]:
# data split in three sets, training, validation and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list].drop(["id"], axis=1)
data_val = data[val_list].drop(["id"], axis=1)
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

Let's upload those data sets in S3

In [28]:
s3_resource = boto3.Session().resource("s3")

train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)
with open(train_file, "rb") as data:
    s3_resource.Bucket(bucket).upload_fileobj(data, os.path.join(prefix, "train", train_file))

validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)
with open(validation_file, "rb") as data:
    s3_resource.Bucket(bucket).upload_fileobj(
        data, os.path.join(prefix, "validation", validation_file)
    )

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
with open(batch_file, "rb") as data:
    s3_resource.Bucket(bucket).upload_fileobj(data, os.path.join(prefix, "batch", batch_file))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
with open(batch_file_noID, "rb") as data:
    s3_resource.Bucket(bucket).upload_fileobj(data, os.path.join(prefix, "batch", batch_file_noID))

---

## Training job and model creation

The below cell uses the [Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_training_job) to kick off the training job using both our training set and validation set. Not that the objective is set to 'binary:logistic' which trains a model to output a probability between 0 and 1 (here the probability of a tumor being malignant).

In [34]:
%%time
from time import gmtime, strftime
from sagemaker.image_uris import retrieve

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)

image = retrieve(framework="xgboost", region=boto3.Session().region_name, version="latest")

output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
print("Training artifacts will be uploaded to: {}".format(output_location))

create_training_params = {
    "AlgorithmSpecification": {"TrainingImage": image, "TrainingInputMode": "File"},
    "RoleArn": role,
    "OutputDataConfig": {"S3OutputPath": output_location},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.m5.4xlarge", "VolumeSizeInGB": 50},
    "TrainingJobName": job_name,
    "HyperParameters": {
        "objective": "binary:logistic",
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.8",
        "silent": "0",
        "num_round": "100",
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 60 * 60},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
            "ContentType": "text/csv",
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
            "ContentType": "text/csv",
        },
    ],
}

sagemaker = boto3.client("sagemaker")
sagemaker.create_training_job(**create_training_params)
status = sagemaker.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
print(status)

try:
    sagemaker.get_waiter("training_job_completed_or_stopped").wait(TrainingJobName=job_name)
finally:
    status = sagemaker.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
    print("Training job ended with status: " + status)
    if status == "Failed":
        message = sagemaker.describe_training_job(TrainingJobName=job_name)["FailureReason"]
        print("Training failed with the following error: {}".format(message))
        raise Exception("Training job failed")

Training job xgb-2023-12-18-16-53-56
Training artifacts will be uploaded to: s3://turin-complete-experiments/breast-cancer/output/xgb-2023-12-18-16-53-56
InProgress
Training job ended with status: Completed
CPU times: user 524 ms, sys: 34.4 ms, total: 559 ms
Wall time: 4min 18s


#### Let's create a model based on our training job. 

The below cell creates a model in SageMaker based on the training job we just executed. The model can later be deployed using the SageMaker hosting services or in our case used in a Batch Transform job.


In [35]:
%%time

model_name = job_name
print(model_name)

info = sagemaker.describe_training_job(TrainingJobName=job_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2023-12-18-16-53-56
arn:aws:sagemaker:us-east-1:898487103080:model/xgb-2023-12-18-16-53-56
CPU times: user 173 ms, sys: 4.01 ms, total: 177 ms
Wall time: 7.1 s


---

## Batch Transform

In SageMaker Batch Transform, we introduced a new attribute called __DataProcessing__.In the below cell, we use the [Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_transform_job) to kick-off several Batch Transform jobs using different configurations of DataProcessing. Please refer to [Associate Prediction Results with Input Records](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to utilize the __DataProcessing__ attribute.




#### 1. Without data processing
Let's first set the data processing fields to null and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [36]:
%%time

from time import gmtime, strftime

batch_job_name = "Batch-Transform-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  # use input data without ID column
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, batch_job_name)

request = {
    "TransformJobName": batch_job_name,
    "ModelName": job_name,
    "TransformOutput": {
        "S3OutputPath": output_location,
        "Accept": "text/csv",
        "AssembleWith": "Line",
    },
    "TransformInput": {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": input_location}},
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None",
    },
    "TransformResources": {"InstanceType": "ml.m4.xlarge", "InstanceCount": 1},
}

sagemaker.create_transform_job(**request)
print("Created Transform job with name: ", batch_job_name)

# Wait until the job finishes
try:
    sagemaker.get_waiter("transform_job_completed_or_stopped").wait(TransformJobName=batch_job_name)
finally:
    response = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    print("Transform job ended with status: " + status)
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")

Created Transform job with name:  Batch-Transform-2023-12-18-17-01-39
Transform job ended with status: Completed
CPU times: user 1.1 s, sys: 43.4 ms, total: 1.15 s
Wall time: 7min 47s


Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [37]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [ ]:
output_df = get_csv_output_from_s3(output_location, batch_file_noID)
output_df.head(8)

#### 2. Join the input and the prediction results 
Now, let's use the new feature to associate the prediction results with their corresponding input records. We can also use the __InputFilter__ to exclude the ID column easily and there's no need to have a separate file in S3.

* Set __InputFilter__ to "$[1:]": indicates that we are excluding column 0 (the 'ID') before processing the inferences and keeping everything from column 1 to the last column (all the features or predictors)  
  
  
* Set __JoinSource__ to "Input": indicates our desire to join the input data with the inference results  


* Leave __OutputFilter__ to default ("$"), indicating that the joined input and inference results be will saved as output.

In [38]:
%%time

batch_job_name = "Batch-Transform-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data with ID column cause InputFilter will filter it out
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, batch_job_name)

request["TransformJobName"] = batch_job_name
request["TransformInput"]["DataSource"]["S3DataSource"]["S3Uri"] = input_location
request["TransformOutput"]["S3OutputPath"] = output_location

request["DataProcessing"] = {
    "InputFilter": "$[1:]",  # exclude the ID column (index 0)
    "JoinSource": "Input",  # join the input with the inference results
}

sagemaker.create_transform_job(**request)
print("Created Transform job with name: ", batch_job_name)

# Wait until the job finishes
try:
    sagemaker.get_waiter("transform_job_completed_or_stopped").wait(TransformJobName=batch_job_name)
finally:
    response = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    print("Transform job ended with status: " + status)
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")

Created Transform job with name:  Batch-Transform-2023-12-18-17-45-39
Transform job ended with status: Completed
CPU times: user 1.1 s, sys: 27.7 ms, total: 1.13 s
Wall time: 6min 42s


Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [39]:
output_df = get_csv_output_from_s3(output_location, batch_file)
output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,84501001,12.460,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,...,40.68,97.65,711.4,0.18530,1.05800,1.10500,0.22100,0.4366,0.20750,0.789157
1,845636,16.020,23.24,102.70,797.8,0.08206,0.06669,0.03299,0.03323,0.1528,...,33.88,123.80,1150.0,0.11810,0.15510,0.14590,0.09975,0.2948,0.08452,0.624410
2,8510653,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.03110,0.1967,...,20.49,96.09,630.5,0.13120,0.27760,0.18900,0.07283,0.3184,0.08183,0.005715
3,855133,14.990,25.20,95.54,698.8,0.09387,0.05131,0.02398,0.02899,0.1565,...,25.20,95.54,698.8,0.09387,0.05131,0.02398,0.02899,0.1565,0.05504,0.020026
4,857155,12.050,14.63,78.04,449.3,0.10310,0.09092,0.06592,0.02749,0.1675,...,20.70,89.88,582.6,0.14940,0.21560,0.30500,0.06548,0.2747,0.08301,0.013134
5,859575,18.940,21.31,123.60,1130.0,0.09009,0.10290,0.10800,0.07951,0.1582,...,26.58,165.90,1866.0,0.11930,0.23360,0.26870,0.17890,0.2551,0.06589,0.989036
6,859711,8.888,14.64,58.79,244.0,0.09783,0.15310,0.08606,0.02872,0.1902,...,15.67,62.56,284.4,0.12070,0.24360,0.14340,0.04786,0.2254,0.10840,0.010582
7,86355,22.270,19.67,152.80,1509.0,0.13260,0.27680,0.42640,0.18230,0.2556,...,28.01,206.80,2360.0,0.17010,0.69970,0.96080,0.29100,0.4055,0.09789,0.992965


#### 3. Update the output filter to keep only ID and prediction results
Let's try to change the output filter  to "$[0,-1]", indicating that when presenting the output, we only want to keep column 0 (the 'ID') and the last column (the inference result i.e. the probability of a given tumor to be malignant)

In [40]:
%%time

batch_job_name = "Batch-Transform-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, batch_job_name)

request["TransformJobName"] = batch_job_name
request["TransformOutput"]["S3OutputPath"] = output_location
request["DataProcessing"][
    "OutputFilter"
] = "$[0, -1]"  # keep the first and last column of the joined output

sagemaker.create_transform_job(**request)
print("Created Transform job with name: ", batch_job_name)

# Wait until the job finishes
try:
    sagemaker.get_waiter("transform_job_completed_or_stopped").wait(TransformJobName=batch_job_name)
finally:
    response = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    print("Transform job ended with status: " + status)
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")

Created Transform job with name:  Batch-Transform-2023-12-18-17-55-59
Transform job ended with status: Completed
CPU times: user 960 ms, sys: 34 ms, total: 994 ms
Wall time: 6min 41s


Now, let's inspect the output of the Batch Transform job in S3 again. It should show 2 columns: the ID and their corresponding probabilities of being malignant.

In [41]:
output_df = get_csv_output_from_s3(output_location, batch_file)
output_df.head(8)

,0,1
0,84501001,0.789157
1,845636,0.624410
2,8510653,0.005715
3,855133,0.020026
4,857155,0.013134
5,859575,0.989036
6,859711,0.010582
7,86355,0.992965


In summary, we can use DataProcessing to 
1. Filter / select useful features from the input dataset. e.g. exclude ID columns.
2. Associate the prediction results with their corresponding input records.
3. Filter the original or joined results before saving to S3. e.g. keep ID and probability columns only.

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker_batch_transform|batch_transform_associate_predictions_with_input|Batch Transform - breast cancer prediction with lowel level SDK.ipynb)
